In [67]:
import os
import pandas as pd
import glob

In [68]:
"""
file format: {model}.{date}.{cycle}.{metric}.csv
create program to compare the forecasted degree days to the observed degree days as well
as the forecast for two days ahead compared to what it is forecasting for the same day the next day
"""

'\nfile format: {model}.{date}.{cycle}.{metric}.csv\ncreate program to compare the forecasted degree days to the observed degree days as well\nas the forecast for two days ahead compared to what it is forecasting for the same day the next day\n'

In [69]:
path = "RawData"

In [70]:
files = glob.glob(path + '/ecmwf-eps.*.pw_cdd.csv')

In [71]:
sorted_files = sorted(files, key=lambda x: (x.split('.')[1], x.split('.')[2]))

In [72]:
errors = []
for i in range(0, len(sorted_files)-1):
    df1 = pd.read_csv(sorted_files[i])
    #get where third column == 1
    forecast = df1[df1.iloc[:, 2] == 1].iloc[0]['Value']
    df2 = pd.read_csv(sorted_files[i+1])
    observation = df2[df2.iloc[:, 2] == 0].iloc[-1]['Value']
    error = forecast - observation
    errors.append(error)


In [75]:
mean_error = sum(errors)/len(errors)

In [76]:
mean_error

-0.02232193196767528